In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
library(googlesheets)
gs_token <- "/home/users/ytanigaw/.googlesheets_token.rds"
gs_auth(token = gs_token)


Auto-refreshing stale OAuth token.


In [3]:
# input
phe_info_f <- '../../common/biomarker.phenotype.info.tsv'
eval_f <- 'snpnet_prs_eval.tsv'
ST_sheet <- 'https://docs.google.com/spreadsheets/d/1j8q1Y7wnMg9nWUm0iT4wJvFfg_hgIXbrtvxelqWHeH4' 

# output
out_f <- 'combined_table.tsv'


In [4]:
phe_info_df <- fread(phe_info_f) %>% rename('Phenotype'='name')

In [5]:
current_tbl_df <- ST_sheet %>% gs_url() %>% 
gs_read(ws = 'ST17')


Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1j8q1Y7wnMg9nWUm0iT4wJvFfg_hgIXbrtvxelqWHeH4
Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Sheet successfully identified: "UKBB Biomarker Paper Supplementary Tables Revisions"
Warning message:
“`as_data_frame()` is deprecated as of tibble 2.0.0.
Please use `as_tibble()` instead.
The signature and semantics have changed, see `?as_tibble`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Accessing worksheet titled 'ST17'.
Warning message:
“Missing column names filled in: 'X8' [8], 'X9' [9]”Parsed with column specification:
cols(
  Trait = col_character(),
  Population = col_character(),
  `Spearman correlation (R) to resid

In [10]:
current_tbl_df <- current_tbl_df[2:nrow(current_tbl_df), colnames(current_tbl_df)[1:6]]

In [45]:
current_tbl_df[,c(2,6)] %>%
group_by(Population) %>%
summarise(relative_r2_to_WB = median(`Variance explained relative to White British`))

Population,relative_r2_to_WB
<chr>,<dbl>
african,0.2385
e_asian,0.5088
non_british_white,0.9282
s_asian,0.7043
white_british,1.0000


In [32]:
eval_df <- fread(eval_f) %>%
rename('trait'='#trait') %>%
select(-annotation) %>%
mutate(pop_split = str_replace(pop_split, 'white_british:test', 'white_british')) %>%
filter(!str_detect(pop_split, ':'))


In [38]:
combined_df <- current_tbl_df %>%
inner_join(
    eval_df,
    by=c('Trait'='trait', 'Population'='pop_split')
) %>%
arrange(Population, Trait)


In [39]:
combined_df %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)


In [40]:
out_f

[1] "combined_table.tsv"